In [1]:
import pandas as pd
from Bio import SeqIO
import glob

In [3]:
df_0 = pd.read_excel(
    "../dataset/AllSatellitesTables.xlsx", sheet_name="PICI", engine="openpyxl"
)
print(df_0.shape)
print(df_0.columns)

In [10]:
metadata = pd.read_csv("../dataset/PICI_GenomicNucInfo_060723.csv")
print(metadata.shape)
print(metadata.columns)

(1435, 6)
Index(['Unnamed: 0', 'PICI_ID', 'Host_ID', 'Host_strain', 'Host_Ref',
       'Genomic_region'],
      dtype='object')


In [4]:
metadata

,Unnamed: 0,PICI_ID,Host_ID,Host_strain,Host_Ref,Genomic_region
0,0,STAU002.0321.00050.C001.PICI.TypeB.variant0002...,STAU002.0321.00050.C001,Staphylococcus aureus strain HOU1444-VR chromo...,GCF_001278745.1,"(401962, 412834)"
1,1,POSP014.0321.00001.C001.PICI.TypeB.variant0001...,POSP014.0321.00001.C001,"Polaromonas sp. JS666, complete sequence.",GCF_000013865.1,"(3126147, 3132812)"
2,2,STAU002.0321.00250.C001.PICI.TypeB.variant0002...,STAU002.0321.00250.C001,Staphylococcus aureus subsp. aureus strain 80w...,GCF_003944865.1,"(814785, 826925)"
3,3,STAU002.0321.00374.C001.PICI.TypeB.variant0002...,STAU002.0321.00374.C001,Staphylococcus aureus strain UP_1150 chromosom...,GCF_009912535.1,"(2745923, 2756343)"
4,4,MYTU002.0321.00238.C001.PICI.TypeB.variant0004...,MYTU002.0321.00238.C001,Mycobacterium tuberculosis strain DKC2 isolate...,GCF_900520315.1,"(105183, 113045)"
...,...,...,...,...,...,...
1430,1430,STSA003.0321.00008.C001.PICI.TypeB.variant0002...,STSA003.0321.00008.C001,Staphylococcus saprophyticus strain UTI-056 ch...,GCF_013341435.1,"(2129, 11546)"
1431,1431,STSI001.0321.00003.C001.PICI.TypeB.variant0001...,STSI001.0321.00003.C001,"Staphylococcus simulans strain MR3, complete s...",GCF_003006055.1,"(1027781, 1037385)"
1432,1432,STSI001.0321.00004.C001.PICI.TypeB.variant0001...,STSI001.0321.00004.C001,"Staphylococcus simulans strain MR4, complete s...",GCF_003006075.1,"(1027781, 1037385)"
1433,1433,STSI001.0321.00005.C001.PICI.TypeB.variant0001...,STSI001.0321.00005.C001,"Staphylococcus simulans strain MR2, complete s...",GCF_003076375.1,"(1027882, 1037486)"


In [22]:
seqs = list(SeqIO.parse("../dataset/PICI_NucSequences_060723.fst", "fasta"))
print(len(seqs))
print("first sample:")
print(seqs[0])

1435
first sample:
ID: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Name: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Description: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Number of features: 0
Seq('TAACTCCCTATCCCCTCTAATTTATTCATCATATCTTGTGCCATTTTATCCGTA...TGA')


In [29]:
seqs_prt_1 = list(
    SeqIO.parse(
        "../dataset/SatelliteProteomes/ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1.prt",
        "fasta",
    )
)
print(len(seqs_prt_1))
print(seqs_prt_1[0])

8
ID: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00
Name: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00
Description: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00 ACER001.0321.00001.C001_00176 C ATG TGA 197540 198730 Valid CDG81_RS00890 1191 @WP_084133721.1@ CDG81_RS00890 1 197540 198730 | site-specific integrase  (translation)
Number of features: 0
Seq('MARVWIEDRSSHSGYKQALAQAKTSRKAAPGRWRVRWYDQEGKPKSQTFSRKPD...DQM')


In [36]:
data_prt = []

prt_files = glob.glob("../dataset/SatelliteProteomes/*.prt")

for prt_file in prt_files:
    sequences = list(SeqIO.parse(prt_file, "fasta"))

    for seq in sequences:
        # Extract function (part after the last '|')
        function = (
            seq.description.split("|")[-1].strip() if "|" in seq.description else ""
        )

        # Extract PICI_ID (everything before the last underscore in Protein_ID)
        pici_id = "_".join(seq.id.split("_")[:-1])

        # Create dictionary for this entry
        entry = {
            "PICI_ID": pici_id,
            "Protein_ID": seq.id,
            "Function": function,
            "Description": seq.description,
        }

        data_prt.append(entry)

df_prt = pd.DataFrame(data_prt)

In [45]:
df_prt.to_csv("../dataset/PICI_proteins.csv", index=False)

In [2]:
df_prt = pd.read_csv("../dataset/PICI_proteins.csv")

In [3]:
df_prt.head()

,PICI_ID,Protein_ID,Function,Description
0,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,site-specific integrase (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
1,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,hypothetical protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
2,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,helix-turn-helix domain-containing protein (t...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
3,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,DUF2742 domain-containing protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
4,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,DUF3631 domain-containing protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...


In [38]:
df_prt["PICI_ID"].unique()

array(['MYTU002.0321.00124.C001.PICI.TypeB.variant0003.Set2',
       'ESCO001.0321.00768.C001.PICI.TypeA.Set2',
       'ESCO001.0321.01362.C001.PICI.TypeA.Set3', ...,
       'ESCO001.0321.01376.C001.PICI.TypeA.Set2',
       'ESCO001.0321.00369.C001.PICI.TypeA.Set2',
       'ESCO001.0321.01394.C001.PICI.TypeA.Set1'],
      shape=(1435,), dtype=object)

In [7]:
print(len(df_prt["PICI_ID"].unique()))
print(len(df_prt["Protein_ID"].unique()))
print(len(df_prt["Function"].unique()))
print(len(df_prt["Description"].unique()))


1435
20594
287
20594


In [11]:
df_prt["Function"].value_counts()

Function
hypothetical protein  (translation)                                     8173
site-specific integrase  (translation)                                   984
terminase small subunit  (translation)                                   797
helix-turn-helix transcriptional regulator  (translation)                764
helix-turn-helix domain-containing protein  (translation)                746
                                                                        ... 
DUF2004 domain-containing protein  (translation)                           1
TrmB family transcriptional regulator  (translation)                       1
antitermination protein  (translation)                                     1
ORF6N domain-containing protein  (translation)                             1
4.4 phage-related function phage major capsid protein  (translation)       1
Name: count, Length: 287, dtype: int64